# Домашнее задние к уроку №3 "Парсинг данных. HTML, Beautiful Soap"

**Собрать информацию о вакансиях на вводимую должность с сайтов [hh.ru](http://hh.ru) и/или [Superjob](http://suberjob.ru) и/или [работа.ру](http://rabota.ru). Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе минимум:** <br>
1. Наименование вакансии.
2. Предлагаемую зарплату (дополнительно: разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
3. Ссылку на саму вакансию.
4. Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с всех сайтов. Общий результат можно вывести с помощью dataFrame через pandas, сохранить в json, либо csv.

In [1]:
#!pip install bs4

In [2]:
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
from time import sleep
import json

In [3]:
finded_job = input()

Наладчик


## HH.RU

In [4]:
# url = 'https://hh.ru/search/vacancy'
# headers = {'User-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:105.0) Gecko/20100101 Firefox/105.0'}
# params = {'text': 'Истопник', 'salary': '' ,'items_on_page': 20, 'enable_snippets': 'true', 'page': 1}

# response = requests.get(url=url, headers=headers, params=params)   

In [5]:
# response.url

In [6]:
# if response.ok:
#     response.encoding = 'utf-8'
#     html = response.content
# else:
#     print('Пришел ответ с кодом {response.status_code}')
#     raise Exception('Ошибка чтения страницы')

In [7]:
# soup = bs(html, 'html.parser')

In [8]:
# found_vacancies = []
# for vac in bs(html, 'html.parser'):
#     cur_vac = {}
#     cur_vac['name'] = vac.find('a', {'class' : 'serp-item__title'}).text
#     cur_vac['companay'] = vac.find('div', {'class': 'vacancy-serp-item__meta-info-company'}).text
#     cur_vac['place'] = vac.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}).text
#     if vac.find('span', {'data-qa' : 'vacancy-serp__vacancy-compensation'}):
#         s = vac.find_all('span', {'data-qa' : 'vacancy-serp__vacancy-compensation'})[0].text
#         cur_vac['salary'] = convert_salary_hhru(s)
#     else:
#         cur_vac['salary'] = None
#     cur_vac['source'] = 'hh.ru'
#     cur_vac['link'] = vac.find('a', {'class':'serp-item__title'})['href']
#     found_vacancies.append(cur_vac)
    
# pprint(found_vacancies)

In [9]:
def convert_salary_hhru(s):
    res = {'string' : s}
    l = s.strip().split()
    if l[0] == 'от':
        res['from'] = int(''.join(filter(lambda x: x.isdigit(), l)))
        res['to'] = None
    elif l[0] == 'до':
        res['from'] = None
        res['to'] = int(''.join(filter(lambda x: x.isdigit(), l)))
    else:
        m = l.index('–')
        res['from'] = int(''.join(l[0:m]))
        res['to'] = int(''.join(filter(lambda x: x.isdigit(), l[m+1:-1])))
    res['valute'] = ''.join(filter(lambda x: not x.isdigit(), l[-2:])).strip('.')
    return res

In [10]:
# Функция будет считытвать не более 200 вакансий
# Функци принимает 
def scrap_hh_ru(find_vacantion: str ='') -> list:
    url: str = 'https://hh.ru/search/vacancy'
    headers: dict = {'User-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:105.0) Gecko/20100101 Firefox/105.0'}
    params: dict = {'text': find_vacantion, 'salary': '' ,'items_on_page': 20, 'enable_snippets': 'true'}
    found_vacancies: list = []
 
    n_vacancies: int = 25
    n: int = 0 
    while n < 10 and n_vacancies > n * 20:
        if n:
            params['page']: int = n
            
        response = requests.get(url=url, headers=headers, params=params)
        if response.ok:
            html: str = response.text
        else:
            print('Пришел ответ с кодом {response.status_code}')
            break
       
        soup = bs(html, 'html.parser')
        
        # блок получения информации о вакансиях.
        # выполняется только при первом вызове считывания страниц
        if n == 0: 
            n_teg = soup.find('h1', {'class': 'bloko-header-section-3'})
            if 'не найдено' in n_teg.text:
                return {}
            n_vacancies = int(''.join(filter(lambda x: x.isdigit(), ''.join(n_teg.text.split('\xa0')).split())))
            
        for vac in soup.find_all('div', {'class': 'serp-item'}):
            cur_vac: dict = {}
            cur_vac['name'] = vac.find('a', {'class' : 'serp-item__title'}).text
            cur_vac['companay'] = vac.find('div', {'class': 'vacancy-serp-item__meta-info-company'}).text
            cur_vac['place'] = vac.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}).text
            if vac.find('span', {'data-qa' : 'vacancy-serp__vacancy-compensation'}):
                s = vac.find_all('span', {'data-qa' : 'vacancy-serp__vacancy-compensation'})[0].text
                cur_vac['salary'] = convert_salary_hhru(s)
            else:
                cur_vac['salary'] = None
            cur_vac['link'] = vac.find('a', {'class':'serp-item__title'})['href']
            cur_vac['source'] = 'hh.ru'
            found_vacancies.append(cur_vac)
        sleep(5)
        n += 1
    return found_vacancies

In [11]:
fv_hh = scrap_hh_ru(finded_job)

Проблемы с кодировками. Нормального решения не нашёл, пришлось "извращаться"


## Работа.ру

In [13]:
# https://rabota.ru/?query=python&sort=relevance&page=2

# url = 'https://rabota.ru/'
# headers = {'User-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:105.0) Gecko/20100101 Firefox/105.0'}
# params = {'query': 'Python'}

# try:
#     response = requests.get(url=url, headers=headers, params=params)
# except Exception as e:
#     print('Ошибка соединения!!!!!')
#     print(e)

In [14]:
# response.url

In [15]:
# if response.ok:
#     response.encoding = 'utf-8'
#     html = response.content
# else:
#     print('Пришел ответ с кодом {response.status_code}')
#     raise Exception('Ошибка чтения страницы')
# response.encoding = 'utf-8'

In [16]:
# soup = bs(html, 'html.parser')

In [17]:
def convert_salary_rabotaru(s):
    if s.lower() != 'по договоренности':
        res = {'string' : s}
        l = s.split()
        if l[0] == 'от':
            res['from'] = int(''.join(filter(lambda x: x.isdigit(), l)))
            res['to'] = None
        elif l[0] == 'до':
            res['from'] = None
            res['to'] = int(''.join(filter(lambda x: x.isdigit(), l)))
        else:
            m = l.index('—')
            res['from'] = int(''.join(l[0:m]))
            res['to'] = int(''.join(filter(lambda x: x.isdigit(), l[m+1:-1])))
        res['valute'] = ''.join(filter(lambda x: not x.isdigit(), l[-2:])).strip('.')
        return res
    else:
        return None

In [19]:
def scrap_rabota_ru(find_vacantion: str ='') -> list:
    
    url = 'https://rabota.ru/'
    headers = {'User-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:105.0) Gecko/20100101 Firefox/105.0'}
    params = {'query': find_vacantion}
    found_vacancies: list = []
 
    n_vacancies: int = 25
    n: int = 0 
    while n < 10 and n_vacancies > n * 20:
        if n:
            params['page']: int = n
        
        try:
            response = requests.get(url=url, headers=headers, params=params)
        except Exception as e:
            print('Ошибка соединения!!!!!')
            print(e)
            return found_vacancies
            
        if response.ok:
            html: str = response.text
        else:
            print('Пришел ответ с кодом {response.status_code}')
            break
       
        soup = bs(html, 'html.parser')
        
        # блок получения информации о вакансиях.
        # выполняется только при первом вызове считывания страниц
        if n == 0: 
            n_teg = soup.find('div', {'class': 'pagination__info hidden-sm-and-down'})
            if n_teg:
                n_vacancies = int(n_teg.text.strip().split()[-1])
            else:
                n_vacancies = 19
            
        for vac in soup.find_all('div', {'class': 'vacancy-preview-card__top'}):
            cur_vac: dict = {}
            cur_vac['name'] = vac.find('a', {'class': 'vacancy-preview-card__title_border'}).text.strip()
            cur_vac['companay'] = vac.find('span', {'class': 'vacancy-preview-card__company-name'}).a.text.strip()
            cur_vac['place'] = vac.find('span', {'class': 'vacancy-preview-location__address-text'}).text.strip()
            s = vac.find('a', {'itemprop': 'title'}).text
            cur_vac['salary'] = convert_salary_rabotaru(s)
            cur_vac['source'] = 'rabota.ru'
            cur_vac['link'] = f"http://rabota.ru{vac.find('a', {'class':'vacancy-preview-card__title_border'})['href']}"
     
            found_vacancies.append(cur_vac)
            
        sleep(55 + n)
        print(n)
        n += 1
    return found_vacancies

In [25]:
fv_rabota = scrap_rabota_ru(finded_job)

0


In [26]:
len(fv_rabota)

10

In [ ]:
# if fv_rfabota:
#     with open('Вакансии_работа.json', mode='w', encoding='UTF-8') as obj:
#         json.dump(fv_rabota, obj, indent=2)

In [27]:
fv_all: list = []
fv_all.extend(fv_hh)
fv_all.extend(fv_rabota)

In [28]:
if fv_all:
    with open('Вакансии.json', mode='w', encoding='UTF-8') as obj:
        json.dump(fv_all, obj, indent=2)


In [29]:
with open('Вакансии.json', mode='r', encoding='UTF-8') as obj:
    v = json.load(obj)
print('Всего сохранено вакансий -', len(v))
print('-' * 100)
pprint(v)


Всего сохранено вакансий - 210
----------------------------------------------------------------------------------------------------
[{'companay': 'ООО\xa0МС ПЛАСТ',
  'link': 'https://hhcdn.ru/click?b=324558&c=40&meta=Jrgybu7_owDk21EfqQsQkwvBVXf8Sm0ofb-vtWj2UxqACVVn74a9MjzhaJ_l3e9YjUO2o77q4HF4a8i6A0QMatyFxWpG4P92HFnHw_7wDpuU4B-SCPTCQOWlI-bLH2k0Th9LxdVvizbxpZcYSi4kwNnrt2Az8S3lq5Cc6YxaUO9QLPi0aPnzMwdXSdK3Kcd1OmTZPWqPbEsUME0gvWB2PQ%3D%3D&place=35&clickType=link_to_vacancy',
  'name': 'Наладчик участка',
  'place': 'Серпухов',
  'salary': {'from': 60000,
             'string': '60\u202f000 – 70\u202f000 руб.',
             'to': 70000,
             'valute': 'руб'},
  'source': 'hh.ru'},
 {'companay': 'АО\xa0ОЧАКОВО',
  'link': 'https://tver.hh.ru/vacancy/67365958?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%9D%D0%B0%D0%BB%D0%B0%D0%B4%D1%87%D0%B8%D0%BA',
  'name': 'Оператор-наладчик производственной линии',
  'place': 'Москва, Говорово  и еще\xa02\xa0',
  'salary': {'fro

  'salary': {'from': 60000,
             'string': '60\u202f000 – 80\u202f000 руб.',
             'to': 80000,
             'valute': 'руб'},
  'source': 'hh.ru'},
 {'companay': 'ООО\xa0Чистая линия',
  'link': 'https://tver.hh.ru/vacancy/69524697?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%9D%D0%B0%D0%BB%D0%B0%D0%B4%D1%87%D0%B8%D0%BA',
  'name': 'Наладчик-слесарь механосборочных работ',
  'place': 'Долгопрудный',
  'salary': {'from': 90000,
             'string': 'от 90\u202f000 руб.',
             'to': None,
             'valute': 'руб'},
  'source': 'hh.ru'},
 {'companay': 'Sintec Group',
  'link': 'https://tver.hh.ru/vacancy/70574634?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%9D%D0%B0%D0%BB%D0%B0%D0%B4%D1%87%D0%B8%D0%BA',
  'name': 'Наладчик технологического оборудования',
  'place': 'Липецк',
  'salary': {'from': 60000,
             'string': '60\u202f000 – 80\u202f000 руб.',
             'to': 80000,
             'valute': 'руб'},
  'sour

 {'companay': 'ООО\xa0Волго-Камская Инжиниринговая Компания',
  'link': 'https://tver.hh.ru/vacancy/70157439?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%9D%D0%B0%D0%BB%D0%B0%D0%B4%D1%87%D0%B8%D0%BA',
  'name': 'Инженер-наладчик АСУТП',
  'place': 'Пермь',
  'salary': {'from': 90000,
             'string': 'от 90\u202f000 руб.',
             'to': None,
             'valute': 'руб'},
  'source': 'hh.ru'},
 {'companay': 'Производство «Фабрика Кухни»',
  'link': 'https://tver.hh.ru/vacancy/70406341?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%9D%D0%B0%D0%BB%D0%B0%D0%B4%D1%87%D0%B8%D0%BA',
  'name': 'Наладчик оборудования',
  'place': 'Москва',
  'salary': {'from': 65000,
             'string': 'от 65\u202f000 руб.',
             'to': None,
             'valute': 'руб'},
  'source': 'hh.ru'},
 {'companay': 'ООО\xa0НИЛСЭ',
  'link': 'https://tver.hh.ru/vacancy/70450411?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%9D%D0%B0%D0%BB%D0%B

             'string': 'от 54\u202f000 руб.',
             'to': None,
             'valute': 'руб'},
  'source': 'hh.ru'},
 {'companay': 'ООО\xa0МЕГАПЛАСТ 2000',
  'link': 'https://tver.hh.ru/vacancy/67929780?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%9D%D0%B0%D0%BB%D0%B0%D0%B4%D1%87%D0%B8%D0%BA',
  'name': 'Наладчик производственного оборудования',
  'place': 'Ногинск',
  'salary': {'from': 47000,
             'string': '47\u202f000 – 75\u202f000 руб.',
             'to': 75000,
             'valute': 'руб'},
  'source': 'hh.ru'},
 {'companay': 'ОАО\xa0НПП Краснознаменец',
  'link': 'https://tver.hh.ru/vacancy/68920337?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%9D%D0%B0%D0%BB%D0%B0%D0%B4%D1%87%D0%B8%D0%BA',
  'name': 'Наладчик холодноштамповочного оборудования',
  'place': 'Санкт-Петербург',
  'salary': {'from': 70000,
             'string': 'от 70\u202f000 руб.',
             'to': None,
             'valute': 'руб'},
  'source': 'hh.ru'},
 